In [175]:
import sys
import os
import numpy as np
import json
import copy
import cv2
from numpy import load
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pickle

# Functions for DWT, Segmnetation, LBP

In [153]:
def get_DWF_image(img):
    # DWT algorithm
    #--------------------#
    img = img.astype(np.float)
    height, width = img.shape[:2]
    result = np.zeros((height, width), np.float)

    # Horizontal processing
    width2 = int(width / 2)
    for i in range(height):
        for j in range(0, width - 1, 2):
            j1 = j + 1
            j2 = int(j / 2)
            result[i, j2] = (img[i, j] + img[i, j1]) / 2
            result[i, width2 + j2] = (img[i, j] - img[i, j1]) / 2
            
    # copy array
    mid_image = np.copy(result)

    # Vertical processing:
    height2 = int(height / 2)
    for i in range(0, height - 1, 2):
        for j in range(0, width):
            i1 = i + 1
            i2 = int(i / 2)
            result[i2, j] = (mid_image[i, j] + mid_image[i1, j]) / 2
            result[height2 + i2, j] = (mid_image[i, j] - mid_image[i1, j]) / 2
    concat_img = result.astype(np.uint8)
    result_img = concat_img[height2:height,0:width2]
    return result_img

In [154]:
def get_segments(img):
    # Split into  3*3 chunks
    #--------------------#
    segments = []
    size = img.shape[0]
    for i in range(0, size-2, 1):
        for j in range(0, size-2, 1):
            segment = img[j:j+3, i:i+3]
            if segment.shape == (3,3):
                segments.append(segment)
    #print("No. of segments: ",len(segments))

    return segments

In [155]:
def get_lbp_values(segments):
    # Apply Local Binary Pattern on individual 3*3 segments in clockwise manner
    #--------------------#
    lbp_values = []
    positions = [[0,0],[0,1],[0,2],[1,2],[2,2],[2,1],[2,0],[1,0]]
    for segment in segments:
        center = segment[1][1]
        value = []
        #print(segment.shape)
        for position in positions:
            if center <= segment[position[0]][position[1]]:
                value.append("1")
            else:
                value.append("0")
        value_binary_string = "".join(value)
        lbp_values.append(int(value_binary_string, 2))
    #print(lbp_values)
    return lbp_values

In [156]:
def euclidean(v1, v2):
    return round(sum((p-q)**2 for p, q in zip(v1, v2)),5) ** 0.5

# Data preparation from npz

In [157]:
data = load("PestData_Train.npz")
X, y = data['arr_0'],data['arr_1']

In [158]:
X[0].shape

(180, 120, 3)

In [159]:
np.mean(y)

0.2833607907742998

In [160]:
img = X[100]
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
result_img = get_DWF_image(img)
segments = get_segments(result_img)
lbp_values = get_lbp_values(segments)
print(len(lbp_values))

5104


In [161]:
df = pd.DataFrame({"Label":list(y.reshape(-1))},columns =["Label"])

In [162]:
df["Vector"] = np.nan
df = df.astype('object')

In [163]:
df.head(20)

,Label,Vector
0,0,NaN
1,0,NaN
2,0,NaN
3,0,NaN
4,0,NaN
5,0,NaN
6,0,NaN
7,0,NaN
8,0,NaN
9,0,NaN


In [164]:
for i in tqdm(range(len(df))):
    img = X[i]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    result_img = get_DWF_image(img)
    segments = get_segments(result_img)
    lbp_values = get_lbp_values(segments)
    df["Vector"][i] = lbp_values

In [165]:
df.head(20)

,Label,Vector
0,0,"[130, 65, 224, 255, 255, 184, 255, 56, 255, 63..."
1,0,"[255, 0, 255, 255, 12, 16, 255, 24, 120, 114, ..."
2,0,"[72, 255, 255, 8, 255, 32, 255, 0, 255, 255, 2..."
3,0,"[255, 63, 5, 204, 18, 255, 255, 255, 255, 255,..."
4,0,"[255, 255, 255, 0, 255, 11, 247, 225, 255, 255..."
5,0,"[255, 255, 176, 255, 255, 255, 255, 255, 166, ..."
6,0,"[255, 255, 255, 255, 255, 255, 255, 255, 0, 25..."
7,0,"[255, 255, 255, 255, 58, 255, 255, 255, 255, 2..."
8,0,"[20, 2, 255, 255, 255, 255, 255, 255, 27, 255,..."
9,0,"[255, 255, 255, 255, 255, 255, 255, 8, 124, 54..."


In [166]:
data = pd.DataFrame(df.Vector.tolist(), index= df.index)
data["Label"] = df["Label"]

In [167]:
data.head(20)

,0,1,2,3,4,5,6,7,8,9,...,5095,5096,5097,5098,5099,5100,5101,5102,5103,Label
0,130,65,224,255,255,184,255,56,255,63,...,255,255,8,92,32,255,255,255,255,0
1,255,0,255,255,12,16,255,24,120,114,...,255,255,255,255,255,255,255,255,255,0
2,72,255,255,8,255,32,255,0,255,255,...,255,255,255,255,2,255,255,32,255,0
3,255,63,5,204,18,255,255,255,255,255,...,60,112,118,3,255,146,103,128,231,0
4,255,255,255,0,255,11,247,225,255,255,...,128,255,255,255,255,255,255,255,0,0
5,255,255,176,255,255,255,255,255,166,0,...,33,255,60,52,100,0,67,255,255,0
6,255,255,255,255,255,255,255,255,0,255,...,255,255,12,88,255,255,31,241,255,0
7,255,255,255,255,58,255,255,255,255,255,...,255,61,170,255,172,0,75,255,168,0
8,20,2,255,255,255,255,255,255,27,255,...,255,255,255,255,255,255,255,255,255,0
9,255,255,255,255,255,255,255,8,124,54,...,142,31,164,70,9,222,40,254,127,0


In [224]:
data.to_csv("DIP_data.csv",index = False)

In [225]:
data = pd.read_csv("DIP_data.csv")

In [226]:
X = data.drop(["Label"],axis = 1)
y = list(data["Label"])

In [227]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [228]:
X.shape

(1821, 5104)

# SVM modelling

In [189]:
from sklearn.svm import SVC

In [171]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [176]:
filename = 'SVM_model.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [178]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6654478976234004


In [195]:
X_test.iloc[0].shape

(5104,)

In [199]:
loaded_model.predict([X_test.iloc[0]])[0]

0

In [179]:
y_pred = loaded_model.predict(X_test)

In [180]:
y_pred[:10]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [181]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[323  59]
 [124  41]]
              precision    recall  f1-score   support

           0       0.72      0.85      0.78       382
           1       0.41      0.25      0.31       165

   micro avg       0.67      0.67      0.67       547
   macro avg       0.57      0.55      0.54       547
weighted avg       0.63      0.67      0.64       547



# Final Model in Use

In [206]:
from keras.preprocessing.image import img_to_array
import cv2 
import pickle

def make_predict_DIP_SVM(image, size):
    X = cv2.resize(image, size) 
    X = img_to_array(X)
    X = np.array(X).reshape(-1, size[0], size[1], 3)
    img = cv2.cvtColor(X[0], cv2.COLOR_BGR2GRAY)
    result_img = get_DWF_image(img)
    segments = get_segments(result_img)
    lbp_values = get_lbp_values(segments)
    loaded_model = pickle.load(open(filename, 'rb'))
    res = loaded_model.predict([lbp_values])[0]
    
    return res


In [223]:
size = (180, 120)

image = cv2.imread("./images/Train_6.jpg", 1) 
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
res = make_predict_DIP_SVM(image, size)

if res == 0:
    print("Not healthy")
else:
    print("Healthy")

Healthy
